In [1]:
import numpy as np
import pandas as pd
import os

In [30]:
df_f_dm =  pd.read_csv("Forecast_demand.csv",parse_dates=["date"])
df_f_r = pd.read_csv("Forecast_Roof.csv",parse_dates=["date"])
df_a_dm = pd.read_csv("Actual_Demand.csv",parse_dates=["date"])
df_a_r = pd.read_csv("Actual_Roof.csv",parse_dates=["date"])
df_train = pd.read_csv("new_train.csv",parse_dates=["date"])

- Align the time, Check the starting/end date
- Giving each forecast period special name, ex:operational 0.5 hour 
- integrate df_train
- solving last changed update time in 5 min interval


In [35]:
dataframes = {
    'Forecast Demand': df_f_dm,
    'Forecast Roof': df_f_r,
    'Actual Demand': df_a_dm,
    'Actual Roof': df_a_r,
    "Train":df_train
}

for name, df in dataframes.items():
    print(f"{name}:")
    print(f"  Start: {df['date'].min()}")
    print(f"  End: {df['date'].max()}")
    print(f"  Total days: {(df['date'].max() - df['date'].min()).days + 1}")
    print()

Forecast Demand:
  Start: 2023-07-08 23:30:00
  End: 2024-08-03 23:00:00
  Total days: 392

Forecast Roof:
  Start: 2023-07-20 00:00:00
  End: 2024-08-21 23:30:00
  Total days: 399

Actual Demand:
  Start: 2023-07-23 00:00:00
  End: 2024-08-17 23:30:00
  Total days: 392

Actual Roof:
  Start: 2023-07-19 23:30:00
  End: 2024-08-21 23:00:00
  Total days: 399

Train:
  Start: 2023-07-20 00:05:00
  End: 2024-09-01 00:00:00
  Total days: 409



In [36]:
start_date = max(df['date'].min() for df in dataframes.values())
end_date = min(df['date'].max() for df in dataframes.values())

print(f"Common date range:")
print(f"  Start: {start_date}")
print(f"  End: {end_date}")
print(f"  Total days: {(end_date - start_date).days + 1}")

Common date range:
  Start: 2023-07-23 00:00:00
  End: 2024-08-03 23:00:00
  Total days: 378


In [37]:
trimmed_dataframes = {}
for name, df in dataframes.items():
    trimmed_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    trimmed_dataframes[name] = trimmed_df
    print(f"{name} trimmed shape: {trimmed_df.shape}")

Forecast Demand trimmed shape: (90705, 21)
Forecast Roof trimmed shape: (90710, 21)
Actual Demand trimmed shape: (90715, 3)
Actual Roof trimmed shape: (90703, 4)
Train trimmed shape: (108853, 3)


In [38]:
for name, df in trimmed_dataframes.items():
    print(f"{name}:")
    print(f"  Start: {df['date'].min()}")
    print(f"  End: {df['date'].max()}")
    print(f"  Total days: {(df['date'].max() - df['date'].min()).days + 1}")
    print()

Forecast Demand:
  Start: 2023-07-23 00:00:00
  End: 2024-08-03 23:00:00
  Total days: 378

Forecast Roof:
  Start: 2023-07-23 00:00:00
  End: 2024-08-03 23:00:00
  Total days: 378

Actual Demand:
  Start: 2023-07-23 00:00:00
  End: 2024-08-03 23:00:00
  Total days: 378

Actual Roof:
  Start: 2023-07-23 00:00:00
  End: 2024-08-03 23:00:00
  Total days: 378

Train:
  Start: 2023-07-23 00:00:00
  End: 2024-08-03 23:00:00
  Total days: 378



In [40]:
for name, df in trimmed_dataframes.items():
    print(name)
    print(len(df))
    df.to_csv(f"./Trim/{name}.csv",index=False)
    

Forecast Demand
90705
Forecast Roof
90710
Actual Demand
90715
Actual Roof
90703
Train
108853


In [41]:
for name, df in trimmed_dataframes.items():
    if 'Region' in df.columns:
        duplicates = df[df.duplicated(subset=['date', 'Region'], keep=False)]
        print(f"{name}:")
        print(f"  Total line: {len(df)}")
        
        print(duplicates)
       
        print()

Forecast Demand:
  總行數: 90705
  重複行數: 0
Empty DataFrame
Columns: [date, Region, 0.5 hour future, 1 hour future, 1.5 hours future, 2 hours future, 2.5 hours future, 3 hours future, 3.5 hours future, 4 hours future, 4.5 hours future, 5 hours future, 5.5 hours future, 6 hours future, 6.5 hours future, 7 hours future, 7.5 hours future, 8 hours future, 8.5 hours future, 9 hours future, 9.5 hours future]
Index: []

[0 rows x 21 columns]
  唯一行數: 90705

Forecast Roof:
  總行數: 90710
  重複行數: 0
Empty DataFrame
Columns: [date, Region, 0.5 hour future, 1 hour future, 1.5 hours future, 2 hours future, 2.5 hours future, 3 hours future, 3.5 hours future, 4 hours future, 4.5 hours future, 5 hours future, 5.5 hours future, 6 hours future, 6.5 hours future, 7 hours future, 7.5 hours future, 8 hours future, 8.5 hours future, 9 hours future, 9.5 hours future]
Index: []

[0 rows x 21 columns]
  唯一行數: 90710

Actual Demand:
  總行數: 90715
  重複行數: 0
Empty DataFrame
Columns: [date, Region, OPERATIONAL_DEMAND]
Inde

### Error data point in Demand Forecast
- 28485 2023-11-04 16:00:00
- 49326 2024-01-30 13:00:00 
- 84609 2024-06-25 13:00:00 

## Import Trim df again

In [70]:
df_f_dm =  pd.read_csv("./Trim/Forecast Demand.csv",parse_dates=["date"])
df_f_r = pd.read_csv("./Trim/Forecast Roof.csv",parse_dates=["date"])
df_a_dm = pd.read_csv("./Trim/Actual Demand.csv",parse_dates=["date"])
df_a_r = pd.read_csv("./Trim/Actual Roof.csv",parse_dates=["date"])
df_train = pd.read_csv("./Trim/Train.csv",parse_dates=["date"])

new_dfs = {
    'Forecast Demand': df_f_dm,
    'Forecast Roof': df_f_r,
    'Actual Demand': df_a_dm,
    'Actual Roof': df_a_r,
    "Train":df_train
}

In [71]:
import pandas as pd
from datetime import timedelta

def check_dataframes(dfs):
    
    all_regions = ['NSW1', 'QLD1', 'VIC1', 'SA1', 'TAS1']

    # 找出所有DataFrame的最早和最晚日期
    all_dates = [df['date'] for df in dfs.values()]
    min_date = min([dates.min() for dates in all_dates])
    max_date = max([dates.max() for dates in all_dates])

    
    full_date_range = pd.date_range(start=min_date, end=max_date, freq='30T')

    for name, df in dfs.items():
        print(f"檢查 {name}:")
        
        # Check missing eate
        missing_dates = set(full_date_range) - set(df['date'])
        if missing_dates:
            print(f"  缺失的日期:")
            for date in sorted(missing_dates):
                print(f"    {date}")

        # ckeck region
        if name == "Train":
            continue
        grouped = df.groupby('date')
        for date, group in grouped:
            missing_regions = set(all_regions) - set(group['Region'])
            if missing_regions:
                print(f"  日期 {date} 缺失的地區:")
                for region in missing_regions:
                    print(f"    {region}")

        print()  


check_dataframes(new_dfs)

檢查 Forecast Demand:
  缺失的日期:
    2023-12-05 13:30:00
    2023-12-28 16:00:00

檢查 Forecast Roof:
  缺失的日期:
    2023-12-05 14:00:00

檢查 Actual Demand:

檢查 Actual Roof:
  缺失的日期:
    2023-12-05 13:00:00
    2023-12-05 13:30:00
  日期 2024-05-28 11:00:00 缺失的地區:
    SA1
  日期 2024-05-28 11:30:00 缺失的地區:
    SA1

檢查 Train:


In [74]:
target = new_dfs["Actual Roof"]
specific_rows = target.loc[target["date"] == "2023-12-05 14:00:00"]
print(specific_rows)

                     date Region    OPOWER          LASTCHANGED
32530 2023-12-05 14:00:00   NSW1  4038.159  2023/12/05 14:19:10
32531 2023-12-05 14:00:00   QLD1  3336.069  2023/12/05 14:19:10
32532 2023-12-05 14:00:00    SA1  1125.039  2023/12/05 14:19:10
32533 2023-12-05 14:00:00   TAS1   201.488  2023/12/05 14:19:10
32534 2023-12-05 14:00:00   VIC1  2124.508  2023/12/05 14:19:10


- 檢查 Forecast Demand:
  缺失的日期:
    2023-12-05 13:30:00
    2023-12-28 16:00:00

- 檢查 Forecast Roof:
  缺失的日期:
    2023-12-05 14:00:00

- 檢查 Actual Demand:

- 檢查 Actual Roof:
  缺失的日期:
    2023-12-05 13:00:00
    2023-12-05 13:30:00
  日期 2024-05-28 11:00:00 缺失的地區:
    SA1
  日期 2024-05-28 11:30:00 缺失的地區:
    SA1

- 檢查 Train:

In [77]:
import pandas as pd
import numpy as np

def fill_forecast_data(df, missing_time):
    
    # df['date'] = pd.to_datetime(df['date'])
    
    
    prev_time = missing_time - pd.Timedelta(minutes=30)
    next_time = missing_time + pd.Timedelta(minutes=30)
    
   
    prev_data = df[df['date'] == prev_time]
    
   
    next_data = df[df['date'] == next_time]
    

    new_rows = []
    
    for region in df['Region'].unique():
        prev_row = prev_data[prev_data['Region'] == region].iloc[0]
        next_row = next_data[next_data['Region'] == region].iloc[0]
        
        new_row = {
            'date': missing_time,
            'Region': region
        }
        

        forecast_columns = [col for col in df.columns if 'future' in col]
        
        
        for i, col in enumerate(forecast_columns[:-1]): 
            next_col = forecast_columns[i+1]
            new_row[col] = prev_row[next_col]
        
        # For 9.5 hours future），use next row's 9 hours future
        last_col = forecast_columns[-1]
        second_last_col = forecast_columns[-2]
        new_row[last_col] = next_row[second_last_col]
        
        new_rows.append(new_row)
    

    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    

    df = df.sort_values(['date', 'Region'])
    
    return df


forecast_demand = new_dfs['Forecast Demand']
forecast_demand = fill_forecast_data(forecast_demand, pd.Timestamp('2023-12-05 13:30:00'))
forecast_demand = fill_forecast_data(forecast_demand, pd.Timestamp('2023-12-28 16:00:00'))


forecast_roof = new_dfs['Forecast Roof']
forecast_roof = fill_forecast_data(forecast_roof, pd.Timestamp('2023-12-05 14:00:00'))


new_dfs['Forecast Demand'] = forecast_demand
new_dfs['Forecast Roof'] = forecast_roof


check_dataframes(new_dfs)

檢查 Forecast Demand:

檢查 Forecast Roof:

檢查 Actual Demand:

檢查 Actual Roof:

檢查 Train:


In [75]:
import pandas as pd
import numpy as np

def fill_missing_data(df):

    # df['date'] = pd.to_datetime(df['date'])
    
    
    all_dates = pd.date_range(df['date'].min(), df['date'].max(), freq='30T')
    all_regions = df['Region'].unique()
    

    full_df = pd.DataFrame([(date, region) for date in all_dates for region in all_regions],
                           columns=['date', 'Region'])
    

    merged_df = pd.merge(full_df, df, on=['date', 'Region'], how='left')
    

    filled_data = []
    
    # Deal every region
    for region in all_regions:
        region_data = merged_df[merged_df['Region'] == region].sort_values('date')
        
        i = 0
        while i < len(region_data):
            row = region_data.iloc[i]
            if pd.isnull(row['OPOWER']):
                # For continues loss
                j = i
                while j < len(region_data) and pd.isnull(region_data.iloc[j]['OPOWER']):
                    j += 1
                
                if i > 0 and j < len(region_data):
                    prev_value = region_data.iloc[i-1]['OPOWER']
                    next_value = region_data.iloc[j]['OPOWER']
                    gap = j - i + 1
                    
                    for k in range(i, j):
                        interpolated_row = region_data.iloc[k].copy()
                        interpolated_row['OPOWER'] = prev_value + (next_value - prev_value) * (k - i + 1) / gap
                        if pd.isnull(interpolated_row['LASTCHANGED']):
                            interpolated_row['LASTCHANGED'] = interpolated_row['date']
                        filled_data.append(interpolated_row)
                    
                    i = j  
                else:
                    filled_data.append(row)
                    i += 1
            else:
                filled_data.append(row)
                i += 1
    

    new_df = pd.DataFrame(filled_data)
    

    new_df = new_df.dropna()
    

    new_df = new_df.sort_values(['date', 'Region'])
    
    return new_df


filled_df = fill_missing_data(new_dfs['Actual Roof'])

new_dfs['Actual Roof'] = filled_df


print(filled_df[(filled_df['date'] == '2023-12-05 13:00:00') | 
                (filled_df['date'] == '2023-12-05 13:30:00') |
                (filled_df['date'] == '2024-05-28 11:00:00') |
                (filled_df['date'] == '2024-05-28 11:30:00')])

                     date Region       OPOWER          LASTCHANGED
32530 2023-12-05 13:00:00   NSW1  4287.297000  2023-12-05 13:00:00
32531 2023-12-05 13:00:00   QLD1  3493.172333  2023-12-05 13:00:00
32532 2023-12-05 13:00:00    SA1   985.937667  2023-12-05 13:00:00
32533 2023-12-05 13:00:00   TAS1   186.708667  2023-12-05 13:00:00
32534 2023-12-05 13:00:00   VIC1  2477.908667  2023-12-05 13:00:00
32535 2023-12-05 13:30:00   NSW1  4162.728000  2023-12-05 13:30:00
32536 2023-12-05 13:30:00   QLD1  3414.620667  2023-12-05 13:30:00
32537 2023-12-05 13:30:00    SA1  1055.488333  2023-12-05 13:30:00
32538 2023-12-05 13:30:00   TAS1   194.098333  2023-12-05 13:30:00
32539 2023-12-05 13:30:00   VIC1  2301.208333  2023-12-05 13:30:00
74510 2024-05-28 11:00:00   NSW1  2751.415000  2024/05/28 11:19:02
74511 2024-05-28 11:00:00   QLD1  1507.219000  2024/05/28 11:19:02
74512 2024-05-28 11:00:00    SA1   709.188667  2024-05-28 11:00:00
74513 2024-05-28 11:00:00   TAS1    61.121000  2024/05/28 11:1

In [56]:
target = new_dfs["Actual Roof"]
specific_rows = target.loc[target["date"] == "2023-12-05 13:00:00"]
print(specific_rows)

Empty DataFrame
Columns: [date, Region, OPOWER, LASTCHANGED]
Index: []


In [79]:
check_dataframes(new_dfs)

檢查 Forecast Demand:

檢查 Forecast Roof:

檢查 Actual Demand:

檢查 Actual Roof:

檢查 Train:


In [78]:
print(filled_df[(filled_df['date'] == '2023-12-05 12:30:00') | 
                (filled_df['date'] == '2023-12-05 14:00:00') |
                (filled_df['date'] == '2024-05-28 11:00:00') |
                (filled_df['date'] == '2024-05-28 11:30:00')])

                     date Region       OPOWER          LASTCHANGED
32525 2023-12-05 12:30:00   NSW1  4411.866000  2023/12/05 12:49:09
32526 2023-12-05 12:30:00   QLD1  3571.724000  2023/12/05 12:49:09
32527 2023-12-05 12:30:00    SA1   916.387000  2023/12/05 12:49:09
32528 2023-12-05 12:30:00   TAS1   179.319000  2023/12/05 12:49:09
32529 2023-12-05 12:30:00   VIC1  2654.609000  2023/12/05 12:49:09
32540 2023-12-05 14:00:00   NSW1  4038.159000  2023/12/05 14:19:10
32541 2023-12-05 14:00:00   QLD1  3336.069000  2023/12/05 14:19:10
32542 2023-12-05 14:00:00    SA1  1125.039000  2023/12/05 14:19:10
32543 2023-12-05 14:00:00   TAS1   201.488000  2023/12/05 14:19:10
32544 2023-12-05 14:00:00   VIC1  2124.508000  2023/12/05 14:19:10
74510 2024-05-28 11:00:00   NSW1  2751.415000  2024/05/28 11:19:02
74511 2024-05-28 11:00:00   QLD1  1507.219000  2024/05/28 11:19:02
74512 2024-05-28 11:00:00    SA1   709.188667  2024-05-28 11:00:00
74513 2024-05-28 11:00:00   TAS1    61.121000  2024/05/28 11:1

In [81]:
for name,df in new_dfs.items():
    print(name)
    print(len(df))
    print(df.columns)

Forecast Demand
90715
Index(['date', 'Region', '0.5 hour future', '1 hour future',
       '1.5 hours future', '2 hours future', '2.5 hours future',
       '3 hours future', '3.5 hours future', '4 hours future',
       '4.5 hours future', '5 hours future', '5.5 hours future',
       '6 hours future', '6.5 hours future', '7 hours future',
       '7.5 hours future', '8 hours future', '8.5 hours future',
       '9 hours future', '9.5 hours future'],
      dtype='object')
Forecast Roof
90715
Index(['date', 'Region', '0.5 hour future', '1 hour future',
       '1.5 hours future', '2 hours future', '2.5 hours future',
       '3 hours future', '3.5 hours future', '4 hours future',
       '4.5 hours future', '5 hours future', '5.5 hours future',
       '6 hours future', '6.5 hours future', '7 hours future',
       '7.5 hours future', '8 hours future', '8.5 hours future',
       '9 hours future', '9.5 hours future'],
      dtype='object')
Actual Demand
90715
Index(['date', 'Region', 'OPERATIONAL_

### Combine Actual/Forecast Demand/Roof together

In [82]:
import pandas as pd


for name, df in new_dfs.items():
    prefix = f"{name.replace(' ', '_')}_"
    df.columns = [prefix + col if col not in ['date', 'Region'] else col for col in df.columns]


merged_df = new_dfs['Forecast Demand']

for name in ['Forecast Roof', 'Actual Demand', 'Actual Roof']:
    merged_df = pd.merge(merged_df, new_dfs[name], 
                         on=['date', 'Region'], 
                         how='outer')


merged_df = merged_df.sort_values(['date', 'Region'])


merged_df = merged_df.reset_index(drop=True)


print(f"rows: {len(merged_df)}")
print(f"cols: {merged_df.shape[1]}")
print(merged_df.columns.tolist())


merged_df.to_csv('merged_data.csv', index=False)


合并后的 DataFrame:
行数: 90715
列数: 43

列名:
['date', 'Region', 'Forecast_Demand_0.5 hour future', 'Forecast_Demand_1 hour future', 'Forecast_Demand_1.5 hours future', 'Forecast_Demand_2 hours future', 'Forecast_Demand_2.5 hours future', 'Forecast_Demand_3 hours future', 'Forecast_Demand_3.5 hours future', 'Forecast_Demand_4 hours future', 'Forecast_Demand_4.5 hours future', 'Forecast_Demand_5 hours future', 'Forecast_Demand_5.5 hours future', 'Forecast_Demand_6 hours future', 'Forecast_Demand_6.5 hours future', 'Forecast_Demand_7 hours future', 'Forecast_Demand_7.5 hours future', 'Forecast_Demand_8 hours future', 'Forecast_Demand_8.5 hours future', 'Forecast_Demand_9 hours future', 'Forecast_Demand_9.5 hours future', 'Forecast_Roof_0.5 hour future', 'Forecast_Roof_1 hour future', 'Forecast_Roof_1.5 hours future', 'Forecast_Roof_2 hours future', 'Forecast_Roof_2.5 hours future', 'Forecast_Roof_3 hours future', 'Forecast_Roof_3.5 hours future', 'Forecast_Roof_4 hours future', 'Forecast_Roof_4.

In [85]:
big_df = pd.read_csv("merged_data.csv",parse_dates=["date"])
big_df

,date,Region,Forecast_Demand_0.5 hour future,Forecast_Demand_1 hour future,Forecast_Demand_1.5 hours future,Forecast_Demand_2 hours future,Forecast_Demand_2.5 hours future,Forecast_Demand_3 hours future,Forecast_Demand_3.5 hours future,Forecast_Demand_4 hours future,...,Forecast_Roof_6.5 hours future,Forecast_Roof_7 hours future,Forecast_Roof_7.5 hours future,Forecast_Roof_8 hours future,Forecast_Roof_8.5 hours future,Forecast_Roof_9 hours future,Forecast_Roof_9.5 hours future,Actual_Demand_OPERATIONAL_DEMAND,Actual_Roof_OPOWER,Actual_Roof_LASTCHANGED
0,2023-07-23 00:00:00,NSW1,8504.0,8280.0,8055.0,7815.0,7560.0,7342.0,7202.0,7114.0,...,0.0,0.774,58.091,337.234,752.700,1209.109,1628.206,8790.0,0.0,2023/07/23 00:19:15
1,2023-07-23 00:00:00,QLD1,5874.0,5718.0,5608.0,5517.0,5450.0,5407.0,5391.0,5390.0,...,0.0,0.889,225.895,613.456,1084.980,1539.251,1930.152,6057.0,0.0,2023/07/23 00:19:15
2,2023-07-23 00:00:00,SA1,1678.0,1656.0,1611.0,1550.0,1480.0,1419.0,1371.0,1337.0,...,0.0,0.000,0.000,0.391,31.420,126.552,255.938,1700.0,0.0,2023/07/23 00:19:15
3,2023-07-23 00:00:00,TAS1,1096.0,1084.0,1078.0,1068.0,1069.0,1071.0,1075.0,1082.0,...,0.0,0.000,0.000,0.561,9.406,26.620,44.738,1109.0,0.0,2023/07/23 00:19:15
4,2023-07-23 00:00:00,VIC1,5303.0,5113.0,4951.0,4780.0,4644.0,4539.0,4476.0,4431.0,...,0.0,0.000,0.000,1.461,95.698,253.389,433.609,5462.0,0.0,2023/07/23 00:19:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90710,2024-08-03 23:00:00,NSW1,8896.0,8668.0,8443.0,8240.0,8028.0,7792.0,7568.0,7379.0,...,0.0,0.000,0.000,0.000,171.185,554.437,1052.269,9140.0,0.0,2024/08/03 23:19:13
90711,2024-08-03 23:00:00,QLD1,6369.0,6194.0,6036.0,5903.0,5791.0,5707.0,5643.0,5596.0,...,0.0,0.000,0.000,56.824,360.462,847.308,1398.471,6512.0,0.0,2024/08/03 23:19:13
90712,2024-08-03 23:00:00,SA1,1559.0,1605.0,1583.0,1580.0,1545.0,1499.0,1443.0,1389.0,...,0.0,0.000,0.000,0.000,0.000,0.000,31.156,1637.0,0.0,2024/08/03 23:19:13
90713,2024-08-03 23:00:00,TAS1,1284.0,1260.0,1242.0,1226.0,1212.0,1202.0,1196.0,1189.0,...,0.0,0.000,0.000,0.000,0.039,0.987,13.910,1325.0,0.0,2024/08/03 23:19:13


In [86]:
QLD_df = big_df[big_df["Region"]=="QLD1"]
QLD_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18143 entries, 1 to 90711
Data columns (total 43 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   date                              18143 non-null  datetime64[ns]
 1   Region                            18143 non-null  object        
 2   Forecast_Demand_0.5 hour future   18143 non-null  float64       
 3   Forecast_Demand_1 hour future     18143 non-null  float64       
 4   Forecast_Demand_1.5 hours future  18143 non-null  float64       
 5   Forecast_Demand_2 hours future    18143 non-null  float64       
 6   Forecast_Demand_2.5 hours future  18143 non-null  float64       
 7   Forecast_Demand_3 hours future    18143 non-null  float64       
 8   Forecast_Demand_3.5 hours future  18143 non-null  float64       
 9   Forecast_Demand_4 hours future    18143 non-null  float64       
 10  Forecast_Demand_4.5 hours future  18143 non-nu